<a href="https://colab.research.google.com/github/Anudeep27-bit/ETL-migration-workflow-Alteryx-to-BigQuery-/blob/main/Prod_Billing_Debits%26_credits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install google-cloud-bigquery


Step 1: Union All of Input Datasets
Step 2: Filter on Description
Step 3: Filter on Amount
Step 4: Country Transformation (Formula)
Step 5: Spend Check
Step 6: Right Join for Negative Spend
Step 7: Final Joins
Step 8: Sorting
Step 9: Tile and Filter
Step 10: Final Output
Each step is now translated into a SQL query and can be run in Google Colab.
The final output is written to a new BigQuery table: final_output_table.

In [5]:
from google.cloud import bigquery
client = bigquery.Client(project='irm-fin-acct-dp-prod')


In [9]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
query = """
Create or Replace TABLE irm-fin-acct-dp-prod.Etl_alt_BQ.billing_detail_snap AS(
SELECT billing_system,  invoicing_country_code, calendar_date,  invoice_date, invoice_id, year_month, transaction_country_name, transaction_cluster,  transaction_region, customer_L1_code, customer_L1_name, imga, billing_cycle,  global_industry_indicator,  sf_country_parent_id, sf_country_parent_name, oracle_account_number,  oracle_account_name,  salesforce_billing_account, bill_code,  description,  product_id, product_summary_type, bill_code_data_source,  gl_account, gl_account_description, revenue_classification, currency, rate, SUM(quantity) as quantity, number_of_months,  SUM(amount) as amount, rate_USD, amount_USD, bill_code_sfdc, unit_of_measure,  storage_from_date,  storage_to_date,  original_invoice_id,  revenue_recognition_date, bill_in_advance_flag, bill_in_advance_month_number, revenue_recognition_amount, allocated_quantity
 FROM `irm-fin-acct-dp-prod.rpt_billing.billing_detail` where (transaction_region = "NA") and transaction_country_name != "Canada" AND calendar_date >= "2023-06-01" and calendar_date <= "2024-08-01" and revenue_classification ="Storage" and product_summary_type = "Records Management"
Group by ALL)
"""
client.query(query).result()
print(" table created successfully.")

In [46]:
query_2 = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.first_filter` AS  -- Changed table name to 'first_filter'
WITH combined_data AS (
  SELECT * FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.billing_detail_snap`
)
SELECT *
FROM combined_data
WHERE description NOT IN ('STORAGE, DESTROYED', 'STORAGE, WITHDRAWN')
  AND description NOT LIKE '%PW\\'D%'  -- Added an extra backslash to escape the single quote
  AND description NOT LIKE '%DEST%';
"""
client.query(query_2).result()  # Run the query and create the table

print("Table created successfully.")

#df_2 = client.query(query_2).to_dataframe()
#print(df_2.head())

Table created successfully.


In [48]:
query_3 = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.second_filter` AS
WITH filtered_data AS (
  SELECT *
  FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.TEST_MAR_2024`
  WHERE description NOT IN ('STORAGE, DESTROYED', 'STORAGE, WITHDRAWN')
    AND description NOT LIKE "%PW'D%"  -- Use double quotes around the string pattern
    AND description NOT LIKE '%DEST%'
)
SELECT *
FROM filtered_data
WHERE amount != 0;
"""
client.query(query_3).result()  # Run the query and create the table

print("Table created successfully.")
#df_3 = client.query(query_3).to_dataframe()
#print(df_3.head())



Table created successfully.


In [50]:
query_4 = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.Third_filter` AS
WITH filtered_data AS (
  SELECT *
  FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.TEST_MAR_2024`
  WHERE description NOT IN ('STORAGE, DESTROYED', 'STORAGE, WITHDRAWN')
    AND description NOT LIKE "%PW'D%"
    AND description NOT LIKE '%DEST%'
    AND amount != 0
),
country_transformed AS (
  SELECT *,
         CASE
           WHEN transaction_country_name = 'US' THEN 'United States'
           ELSE transaction_country_name
         END AS Country -- Create a new column 'Country'
  FROM filtered_data
)
SELECT *
FROM country_transformed;
"""

client.query(query_4).result()  # Run the query and create the table

print("Table created successfully.")

#df_4 = client.query(query_4).to_dataframe()
#print(df_4.head())


Table created successfully.


In [66]:
# Query 1: Create table for negative amounts with "Cycle" column
query_negative = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.negative_amounts` AS
SELECT *,
       'credits' AS Cycle  -- Adding "Cycle" column with value "credits"
FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.Third_filter`
WHERE amount < 0;
"""

# Query 2: Create table for non-negative amounts (unchanged)
query_non_negative = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.non_negative_amounts` AS
SELECT *
FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.Third_filter`
WHERE amount >= 0;
"""

# Execute both queries
client.query(query_negative).result()
client.query(query_non_negative).result()

print("Tables created successfully.")


Tables created successfully.


In [67]:
query_summary = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.Summary_negative_amt` AS
SELECT
    `Year_Month`,
    `customer_L1_code`,
    -`amount` AS amount,  -- Negate the amount values
    COUNT(*) AS Count
FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.negative_amounts`
GROUP BY `Year_Month`, `customer_L1_code`, `amount`
ORDER BY `Year_Month`, `customer_L1_code`, `amount`;



"""

client.query(query_summary).result()  # Execute the query to create the summary table
print("Summary table created successfully.")



Summary table created successfully.


In [68]:
query_inner_join = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.joined_inner_amounts` AS
SELECT
    non_neg.`Year_Month`,
    non_neg.`customer_L1_code`,
    non_neg.`amount` AS non_negative_amount,
    neg.`amount` AS negative_amount,
    'debits' AS Cycle  -- Adding "Cycle" column with "debits" value
FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.non_negative_amounts` AS non_neg
JOIN `irm-fin-acct-dp-prod.Etl_alt_BQ.negative_amounts` AS neg
ON non_neg.`Year_Month` = neg.`Year_Month`
AND non_neg.`customer_L1_code` = neg.`customer_L1_code`
AND non_neg.`amount` = neg.`amount`;
"""

client.query(query_inner_join).result()  # Execute the query to create the inner join table with the Cycle column
print("Inner joined table with Cycle column created successfully.")




Inner joined table with Cycle column created successfully.


In [85]:
from google.cloud import bigquery

# Initialize BigQuery client with project ID
client = bigquery.Client(project="irm-fin-acct-dp-prod")

query = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.merged_table` AS
SELECT
    COALESCE(t1.Cycle, t2.Cycle) AS Cycle,
    COALESCE(t1.customer_L1_code, t2.customer_L1_code) AS customer_L1_code,
    t1.Year_Month,
    t1.negative_amount,
    t1.non_negative_amount,
    t2.bill_in_advance_flag,
    t2.currency,
    t2.oracle_account_number,
    t2.transaction_region,
    t2.Country,
    t2.transaction_cluster,
    t2.bill_in_advance_month_number,
    t2.oracle_account_name,
    t2.sf_country_parent_id,
    t2.storage_to_date,
    t2.allocated_quantity,
    t2.global_industry_indicator,
    t2.imga,
    t2.quantity,
    t2.invoice_date,
    t2.revenue_classification,
    t2.product_summary_type,
    t2.bill_code,
    t2.amount_USD,
    t2.number_of_months,
    t2.bill_code_sfdc,
    t2.revenue_recognition_date,
    t2.rate_USD,
    t2.customer_L1_name,
    t2.invoicing_country_code,
    t2.sf_country_parent_name,
    t2.invoice_id,
    t2.description,
    t2.gl_account,
    t2.billing_system,
    t2.bill_code_data_source,
    t2.unit_of_measure,
    t2.original_invoice_id,
    t2.year_month AS year_month_t2,  # Rename year_month from t2 to avoid conflict
    t2.revenue_recognition_amount,
    t2.transaction_country_name,
    t2.billing_cycle,
    t2.gl_account_description,
    t2.amount AS amount_t2, # Rename amount from t2 to avoid conflict
    t2.rate,
    t2.storage_from_date,
    t2.product_id,
    t2.salesforce_billing_account,
    t2.calendar_date
FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.joined_inner_amounts` AS t1
FULL OUTER JOIN `irm-fin-acct-dp-prod.Etl_alt_BQ.negative_amounts` AS t2
ON t1.Cycle = t2.Cycle
AND t1.customer_L1_code = t2.customer_L1_code;
"""

client.query(query).result()

print("Merged table created successfully.")

Merged table created successfully.


In [92]:
query = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.summarized_table` AS
SELECT
    Cycle,  -- Group by Cycle
    calendar_date AS `Transaction Date`,  -- Group by calendar_date, alias it as 'Transaction Date'
    Country,  -- Group by Country
    customer_L1_code AS CustID,  -- Group by customer_L1_code, alias it as CustID
    oracle_account_number AS `Oracle ID`,  -- Group by oracle_account_number, alias it as 'Oracle ID'
    ANY_VALUE(oracle_account_name) AS `Customer Name`,  -- Use ANY_VALUE to select a value without grouping
    SUM(amount_t2) AS Amount  -- Sum the Spend column
FROM
    `irm-fin-acct-dp-prod.Etl_alt_BQ.merged_table`
GROUP BY
    Cycle, calendar_date, Country, customer_L1_code, oracle_account_number;
"""

client.query(query).result()
print("Summarized table created successfully.")


Summarized table created successfully.
